In [1]:
!pip install 'gluonts[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.5 MB/s eta 0:00:00
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 2.2.2
    Uninstalling pytorch-lightning-2.2.2:
      Successfully uninstalled pytorch-lightning-2.2.2


In [2]:
!pip install optuna

In [3]:
import pandas as pd
import numpy as np
from gluonts.dataset.pandas import PandasDataset
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.torch.distributions import NormalOutput
from gluonts.evaluation import make_evaluation_predictions
import matplotlib.pyplot as plt
import json


path = '../input/tourism-data/'
# Split train and test
df = pd.read_csv(path+'Tourism_process_for_deepar.csv')
df.set_index('Date',inplace=True)
#df.index = pd.to_datetime(df.index).dt.strftime('%Y-%m-%d %H:%M:%S')
prediction_length = 12
freq = 'MS'
split_date = pd.to_datetime('2016-01-01').strftime('%Y-%m-%d %H:%M:%S')
train = df[df.index < split_date]
test = df[df.index >= split_date]

In [4]:
split_date1 = pd.to_datetime('2015-01-01').strftime('%Y-%m-%d %H:%M:%S')
train1 = df[df.index < split_date1]
val = train[train.index>=split_date1]

In [5]:
train1

,Node,Value,Month_Of_Year,Region,Zone,State
Date,,,,,,
1998-01-01 00:00:00,T,45151.071280,1,0,0,0
1998-01-01 00:00:00,A,17515.502380,1,0,0,1
1998-01-01 00:00:00,B,10393.618016,1,0,0,2
1998-01-01 00:00:00,C,8633.359047,1,0,0,3
1998-01-01 00:00:00,D,3504.313346,1,0,0,4
...,...,...,...,...,...,...
2014-12-01 00:00:00,GAC,29.792837,12,72,26,7
2014-12-01 00:00:00,GBA,30.358611,12,73,27,7
2014-12-01 00:00:00,GBB,1.530844,12,74,27,7


In [6]:
train1.reset_index(inplace=True)
val.reset_index(inplace=True)

train2 = train1.drop_duplicates(subset=['Node','Region','Zone','State'])
train_static = pd.DataFrame({'State':train2['State'],
                             'Zone':train2['Zone'],
                             'Region':train2['Region'],
                             'Node':train2['Node']})
train_static.set_index('Node',inplace=True)

train_group = train1.groupby('Node')
standardized_params = {}
train_standard = train1.copy()

# 标准化
for cat,group in train_group:
    means = group['Value'].mean()
    stds = group['Value'].std()
    standardized_params[cat] = {'mean':means,'std':stds}
    train_standard.loc[group.index,'Value'] = (group['Value']-means)/stds


In [7]:
train_standard

,Date,Node,Value,Month_Of_Year,Region,Zone,State
0,1998-01-01 00:00:00,T,3.274775,1,0,0,0
1,1998-01-01 00:00:00,A,3.854917,1,0,0,1
2,1998-01-01 00:00:00,B,3.319479,1,0,0,2
3,1998-01-01 00:00:00,C,1.583920,1,0,0,3
4,1998-01-01 00:00:00,D,3.559008,1,0,0,4
...,...,...,...,...,...,...,...
22639,2014-12-01 00:00:00,GAC,-0.589784,12,72,26,7
22640,2014-12-01 00:00:00,GBA,0.742826,12,73,27,7
22641,2014-12-01 00:00:00,GBB,-1.003333,12,74,27,7
22642,2014-12-01 00:00:00,GBC,-0.224970,12,75,27,7


In [8]:
train_ds = PandasDataset.from_long_dataframe(train_standard.iloc[:,[0,1,2,3]],
                                             target="Value",
                                             timestamp='Date',
                                             freq='M',
                                             item_id="Node",
                                             feat_dynamic_real=["Month_Of_Year"],
                                             static_features=train_static)

In [9]:
val

,Date,Node,Value,Month_Of_Year,Region,Zone,State
0,2015-01-01 00:00:00,T,44072.739245,1,0,0,0
1,2015-01-01 00:00:00,A,14467.899039,1,0,0,1
2,2015-01-01 00:00:00,B,9989.135051,1,0,0,2
3,2015-01-01 00:00:00,C,8928.542621,1,0,0,3
4,2015-01-01 00:00:00,D,3705.183969,1,0,0,4
...,...,...,...,...,...,...,...
1327,2015-12-01 00:00:00,GAC,20.828564,12,72,26,7
1328,2015-12-01 00:00:00,GBA,11.350454,12,73,27,7
1329,2015-12-01 00:00:00,GBB,0.000000,12,74,27,7
1330,2015-12-01 00:00:00,GBC,109.454456,12,75,27,7


In [10]:
val_group = val.groupby('Node')
val_standard = val.copy()

# 标准化
for cat,group in val_group:
    means = group['Value'].mean()
    stds = group['Value'].std()
    val_standard.loc[group.index,'Value'] = (group['Value']-means)/stds

In [11]:
val_standard

,Date,Node,Value,Month_Of_Year,Region,Zone,State
0,2015-01-01 00:00:00,T,2.950906,1,0,0,0
1,2015-01-01 00:00:00,A,2.924239,1,0,0,1
2,2015-01-01 00:00:00,B,2.978696,1,0,0,2
3,2015-01-01 00:00:00,C,1.889177,1,0,0,3
4,2015-01-01 00:00:00,D,2.854171,1,0,0,4
...,...,...,...,...,...,...,...
1327,2015-12-01 00:00:00,GAC,-0.763644,12,72,26,7
1328,2015-12-01 00:00:00,GBA,-0.168195,12,73,27,7
1329,2015-12-01 00:00:00,GBB,-1.302586,12,74,27,7
1330,2015-12-01 00:00:00,GBC,0.297599,12,75,27,7


In [19]:
estimator = DeepAREstimator(
            num_layers=3,
            hidden_size=40,
            lr=1e-3,
            prediction_length=12,
            context_length=10*prediction_length,
            patience = 10,
            freq='MS',
            distr_output=NormalOutput(),
            scaling=False,
            num_parallel_samples=1000,
            batch_size=32,
            trainer_kwargs={
                "max_epochs": 20,
            },
        )

predictor = estimator.train(train_ds)
forecast_it = predictor.predict(train_ds)

forecasts = list(forecast_it)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
2024-05-02 08:16:34.330860: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 08:16:34.330990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 08:16:34.461893: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO: LOCAL

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 1.01100 (best 1.01100), saving model to '/kaggle/working/lightning_logs/version_0/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 0.89567 (best 0.89567), saving model to '/kaggle/working/lightning_logs/version_0/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 0.83817 (best 0.83817), saving model to '/kaggle/working/lightning_logs/version_0/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 0.79543 (best 0.79543), saving model to '/kaggle/working/lightning_logs/version_0/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 0.76394 (best 0.76394), saving model to '/kaggle/working/lightning_logs/version_0/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 0.74717 (best 0.74717), saving model to '/kaggle/worki

In [21]:
forecasts[0].samples

array([[ 2.757091  , -0.8080652 , -1.0843526 , ...,  0.42752808,
        -0.6124935 , -0.72810507],
       [ 3.0293152 , -0.6537782 , -0.5317347 , ...,  0.66828775,
        -0.40513134, -0.16052985],
       [ 2.580253  , -0.7947694 , -0.4406939 , ...,  0.3909461 ,
        -0.44125494,  0.18742868],
       ...,
       [ 1.7074896 , -0.3810192 , -1.0399537 , ..., -0.04010795,
         0.27660513, -0.64579606],
       [ 2.2419195 , -0.5966748 , -0.00652245, ...,  0.49082142,
        -0.6202163 , -0.63950914],
       [ 2.4376235 , -0.6919148 , -0.42613304, ...,  0.73359096,
        -0.38487917, -0.13849175]], dtype=float32)

In [27]:
node_nonsort = []
for i in range(len(forecasts)):
    node_nonsort.append(forecasts[i].item_id)
new_df = pd.read_csv(path+'/Tourism_process.csv')
node_list = new_df.columns[2:].tolist()

In [29]:
index_list = [node_nonsort.index(i) for i in node_list]

In [33]:
forecasts[0].samples[:,0].shape

(1000,)

In [74]:
samples_total = []
for j in range(12):
    samples_step = []
    for i in range(len(forecasts)):
        samples_step.append(forecasts[i].samples[:,j].reshape((1,1000)))
    samples_1 = []
    for k in index_list:
        samples_1.append(samples_step[k])
    samples_total.append(np.concatenate(samples_1,axis=0))


In [75]:
samples_new = np.concatenate(samples_total,axis=1)

In [130]:
samples_new

(111, 12000)

In [135]:
samples_new2 = samples_new.copy()
samples_new2 = samples_new2.transpose()
np.random.shuffle(samples_new2)
samples_new2 = samples_new2.transpose()

In [137]:
samples_new2

array([[-6.2894499e-01,  2.7472367e+00,  4.9591601e-01, ...,
         3.3758691e-01,  4.3860447e-01, -5.2227789e-01],
       [-7.8615975e-01,  2.4988220e+00,  7.4630612e-01, ...,
        -5.6354260e-01,  1.0224484e+00, -7.8539824e-01],
       [ 3.7997833e-01,  3.1360981e+00,  2.3612043e-01, ...,
        -3.5828272e-01,  1.2187922e-01, -8.0225033e-01],
       ...,
       [-1.1683521e+00,  3.7436658e-01, -4.5901632e-01, ...,
        -3.3158427e-01, -5.8031994e-01, -1.2946153e+00],
       [ 2.2954570e-01, -1.2408385e+00, -4.5528954e-01, ...,
         5.8066189e-01,  6.3842344e-01,  1.8118092e+00],
       [-4.4324553e-01, -1.7809411e+00, -1.5722710e+00, ...,
        -3.0964091e-03,  7.7119803e-01, -1.5302658e-02]], dtype=float32)

array([[5, 6],
       [1, 0],
       [3, 4]])

In [109]:
real = []
for i in range(12):
    real_step = np.array(val_standard.iloc[i*111:(i+1)*111,2].tolist())
    repeated_arr = np.repeat(real_step, 1000, axis=0)
    two_dim_array = repeated_arr.reshape(111, 1000).transpose()
    real.append(two_dim_array)
real_2 = np.concatenate(real,axis=0).transpose()
        

In [104]:
val_standard.tail(111)

,Date,Node,Value,Month_Of_Year,Region,Zone,State
1221,2015-12-01 00:00:00,T,-0.218299,12,0,0,0
1222,2015-12-01 00:00:00,A,-0.373919,12,0,0,1
1223,2015-12-01 00:00:00,B,0.061718,12,0,0,2
1224,2015-12-01 00:00:00,C,-0.628933,12,0,0,3
1225,2015-12-01 00:00:00,D,0.231343,12,0,0,4
...,...,...,...,...,...,...,...
1327,2015-12-01 00:00:00,GAC,-0.763644,12,72,26,7
1328,2015-12-01 00:00:00,GBA,-0.168195,12,73,27,7
1329,2015-12-01 00:00:00,GBB,-1.302586,12,74,27,7
1330,2015-12-01 00:00:00,GBC,0.297599,12,75,27,7


In [113]:
real_2.shape

(111, 12000)

In [141]:

class DeepARTuningObjective:
    def __init__(
        self, dataset, prediction_length, freq, validation_label,index_list
    ):
        self.dataset = dataset
        self.prediction_length = prediction_length
        self.freq = freq
        self.validation_label=validation_label


    def get_params(self, trial) -> dict:
        return {
            "num_layers": trial.suggest_int("num_layers", 2, 5),
            "hidden_size": trial.suggest_int("hidden_size", 20, 50),
            "lr":trial.suggest_categorical('lr', choices=[1e-4,1e-3,1e-2]),
            "batch_size": trial.suggest_int("batch_size", low= 16, high =64, step =16)
        }

    def __call__(self, trial):
        params = self.get_params(trial)
        estimator = DeepAREstimator(
            num_layers=params["num_layers"],
            hidden_size=params["hidden_size"],
            lr=params["lr"],
            prediction_length=self.prediction_length,
            context_length=10*prediction_length,
            patience = 10,
            freq=self.freq,
            distr_output=NormalOutput(),
            scaling=False,
            num_parallel_samples=1000,
            batch_size=params["batch_size"],
            trainer_kwargs={
                "max_epochs": 20,
            },
        )

        predictor = estimator.train(self.dataset, cache_data=True)
        forecast_it = predictor.predict(self.dataset)

        forecasts = list(forecast_it)
        
        samples_total = []
        for j in range(12):
            samples_step = []
            for i in range(len(forecasts)):
                samples_step.append(forecasts[i].samples[:,j].reshape((1,1000)))
            samples_1 = []
            for k in index_list:
                samples_1.append(samples_step[k])
            samples_total.append(np.concatenate(samples_1,axis=0))
        samples_new = np.concatenate(samples_total,axis=1)
        
        real = []
        for i in range(12):
            real_step = np.array(val_standard.iloc[i*111:(i+1)*111,2].tolist())
            repeated_arr = np.repeat(real_step, 1000, axis=0)
            two_dim_array = repeated_arr.reshape(111, 1000).transpose()
            real.append(two_dim_array)
        real_2 = np.concatenate(real,axis=0).transpose()
        
        samples_new2 = samples_new.copy()
        samples_new2 = samples_new2.transpose()
        np.random.shuffle(samples_new2)
        samples_new2 = samples_new2.transpose()
        
        dif1 = samples_new - samples_new2
        dif2 = real_2 - samples_new
        term1 = np.sum(np.square(dif1))
        term2 = np.sum(np.square(dif2))
        res1 = ((-0.5*term1)+term2)/(real_2.shape[1])
        
        return res1

In [142]:
import time
import optuna

start_time = time.time()
study = optuna.create_study(direction="minimize")
study.optimize(
    DeepARTuningObjective(
        train_ds, prediction_length, freq,val_standard, index_list
    ),
    n_trials=10,
)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
print(time.time() - start_time)

[I 2024-05-02 09:13:32,926] A new study created in memory with name: no-name-0c32630b-bf9e-45db-b731-e9bce4e161af
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type        | Params | In sizes                                                      | Out sizes    
----------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 13.1 K | [[1, 1], [1, 1], [1, 156, 2], [1, 156], [1, 156], [1, 12, 2]] | [1, 1000, 12]
----------------------------------------------------------------------------------------------------------------------
13.1 K    Tra

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 1.08082 (best 1.08082), saving model to '/kaggle/working/lightning_logs/version_1/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 1.04635 (best 1.04635), saving model to '/kaggle/working/lightning_logs/version_1/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 1.03543 (best 1.03543), saving model to '/kaggle/working/lightning_logs/version_1/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 1.01736 (best 1.01736), saving model to '/kaggle/working/lightning_logs/version_1/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 1.00309 (best 1.00309), saving model to '/kaggle/working/lightning_logs/version_1/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 0.98351 (best 0.98351), saving model to '/kaggle/worki

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 0.92368 (best 0.92368), saving model to '/kaggle/working/lightning_logs/version_2/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 0.79145 (best 0.79145), saving model to '/kaggle/working/lightning_logs/version_2/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 0.73152 (best 0.73152), saving model to '/kaggle/working/lightning_logs/version_2/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 0.71114 (best 0.71114), saving model to '/kaggle/working/lightning_logs/version_2/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 0.65685 (best 0.65685), saving model to '/kaggle/working/lightning_logs/version_2/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 0.64084 (best 0.64084), saving model to '/kaggle/worki

...

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 1.03778 (best 1.03778), saving model to '/kaggle/working/lightning_logs/version_9/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 0.91861 (best 0.91861), saving model to '/kaggle/working/lightning_logs/version_9/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 0.84682 (best 0.84682), saving model to '/kaggle/working/lightning_logs/version_9/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 0.80668 (best 0.80668), saving model to '/kaggle/working/lightning_logs/version_9/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 0.75463 (best 0.75463), saving model to '/kaggle/working/lightning_logs/version_9/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 0.75270 (best 0.75270), saving model to '/kaggle/worki

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 0.95515 (best 0.95515), saving model to '/kaggle/working/lightning_logs/version_10/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 0.85411 (best 0.85411), saving model to '/kaggle/working/lightning_logs/version_10/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 0.80133 (best 0.80133), saving model to '/kaggle/working/lightning_logs/version_10/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 0.78159 (best 0.78159), saving model to '/kaggle/working/lightning_logs/version_10/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 0.74517 (best 0.74517), saving model to '/kaggle/working/lightning_logs/version_10/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 0.70355 (best 0.70355), saving model to '/kaggle/

Number of finished trials: 10
Best trial:
  Value: 10.073001049791909
  Params: 
    num_layers: 3
    hidden_size: 41
    lr: 0.01
    batch_size: 16
182.81919741630554
